<a href="https://colab.research.google.com/github/wai-ming-chan/fed_avg/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic classifier with TensorFlow


This a series of three tutorials you are in the first one: 
* [Basic Classifier](https://github.com/coMindOrg/federated-averaging-tutorials/blob/master/Basic%20Classifier.ipynb)
* [Basic Distributed Classifier](https://github.com/coMindOrg/federated-averaging-tutorials/blob/master/Basic%20Distributed%20Classifier.ipynb)
* [Basic Federated Classifier](https://github.com/coMindOrg/federated-averaging-tutorials/blob/master/Basic%20Federated%20Classifier.ipynb)

## What is TensorFlow?

TensorFlow™ is an open source library developed by Google, for high performance numerical computation which has become mainstream within the Deep Learning community. The basic architecture of Tensorflow could be summarized as follows: 

- Operations are defined (layers of the neural network, sums, multiplications, optimizers ...) as nodes of a graph. 

- The data is in the form of a tensor. These tensors go through the nodes of the graph, where the defined operations are executed on them (hence the name TensorFlow). 

- Once the graph and the input tensors have been defined, the entire process described above is executed in a Session.

If you would like to get a better idea of how TensorFlow works, we highly reccomed you to read the following article:

<a href="https://medium.com/@camrongodbout/tensorflow-in-a-nutshell-part-one-basics-3f4403709c9d">TensorFlow in a nutshell</a>

You could also refer to the <a href="https://www.tensorflow.org">official TensorFlow web page</a> for more information, tutorials and instalation guides.

This basic classifier tutorial is based on the official TensorFlow tutorial, we have added some additional concepts such as the Datasets API and the Monitored Training Session. This is because we will need those concepts in the more advanced tutorials and we hope showing them here will make the series easier to follow.

## Why only TensorFlow and not keras?

TensorFlow incorporates a high level module called keras. It simplifies the usage a TensorFlow to a great extent.

These tutorials could also use Keras, but native TensorFlow is more transparent allowing us to demonstrate more easily all the steps and decisions that are taken in the creation of the graph. We will only use Keras to import the data and define the cost function for convenience.

## First step: import libraries

The first one to be imported will be tensorflow, which by agreement is usually given the abbreviated name of tf.

Although we really do not need to do to import keras (we could use it from tf.keras) we will also import it separately for convenience.

Finally, three traditional basic Python libraries: numpy, matplotlib and time.

In the next cell, we also define some basic constant variables for our algorithm. The number of epochs to train and the size of the batches we are going to use for trainning.

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
BATCH_SIZE = 32
EPOCHS = 5

## Load and normalize the data

We are going to work with a database called fashion mnist kindly put together by Zalando to substitute the more traditional handwritten digit database. It contains photographs consisting of 28x28 pixels in black and white of 10 different types of clothing. The features on which we will train our model will be the values of each pixel, ranging from 0 to 255.

After loading we will divide these values by 255 so that they are normalized between 0 and 1.

We have also added a list with the names of the different types of clothing, so that we can visualize the images with their names later on.

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print('Data loaded')
print('Local dataset size: {}'.format(train_images.shape[0]))

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0
test_images = test_images / 255.0

4422102/4422102 [==============================] - 0s 0us/step
Data loaded
Local dataset size: 60000


## Global Step

Next we create a global step for the optimizer. This is a variable that increases every time the parameters of the model are updated. It is not necessary to create it always in TensorFlow, and it could also be created as a normal variable. However, by creating it through this function we guarantee that any other function that requires the global step will find it through its standardized name (a name that the variable carries below, its context or name scope). In addition, the function checks if a global step already exists, and it associates it with this variable instead of creating a new one.

In [6]:
# global_step = tf.train.get_or_create_global_step()
# global_step = tf.training_util.get_or_create_global_step()
global_step = tf.compat.v1.train.get_or_create_global_step()